In [1]:
# a bunch of import statements
import numpy as np
import pandas as pd
import os
import sys
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from plotting import normalize_image, plot_image_array, plot_confusion_matrix, plot_model_history
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# set various paths
data_path = os.path.join(sys.path[0], 'galaxy-zoo-the-galaxy-challenge/data')
training_path = os.path.join(data_path, 'images_training_rev1')
test_path = os.path.join(data_path, 'images_test_rev1')

# find file names
training_file_names = os.listdir(training_path)
test_file_names = os.listdir(test_path)

In [ ]:
# load csv data
all_ones_benchmark = pd.read_csv(os.path.join(data_path, 'all_ones_benchmark.csv'), dtype=str)
all_zeros_benchmark = pd.read_csv(os.path.join(data_path, 'all_zeros_benchmark.csv'), dtype=str)
central_pixel_benchmark = pd.read_csv(os.path.join(data_path, 'central_pixel_benchmark.csv'), dtype=str)
training_solutions_rev1 = pd.read_csv(os.path.join(data_path, 'training_solutions_rev1.csv'), dtype=str)

In [5]:
# For Evelyn
training_path = os.path.join(sys.path[0], 'images_training_rev1')
test_path = os.path.join(sys.path[0], 'images_test_rev1')
training_file_names = os.listdir(training_path)
test_file_names = os.listdir(test_path)


all_ones_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_ones_benchmark.csv'), dtype=str)
all_zeros_benchmark = pd.read_csv(os.path.join(sys.path[0], 'all_zeros_benchmark.csv'), dtype=str)
central_pixel_benchmark = pd.read_csv(os.path.join(sys.path[0], 'central_pixel_benchmark.csv'), dtype=str)
training_solutions_rev1 = pd.read_csv(os.path.join(sys.path[0], 'training_solutions_rev1.csv'), dtype=str)

In [6]:
# load training images + sol
training_img = []
training_img_sol = []

for file in training_file_names[:1500]: 
    # extract training image
    training_img.append(cv2.imread(os.path.join(training_path, file)))
    
    # extract corresponding galaxy class probabilities from answer key
    galaxy_id = file[:6]
    sol_data = training_solutions_rev1.loc[training_solutions_rev1['GalaxyID'] == galaxy_id]
    training_img_sol.append(np.array(sol_data.iloc[0].tolist()[1:]))

# convert to numpy arrays
training_img = np.array(training_img)
training_img_sol = np.array(training_img_sol)

# load testing images
test_img = []
for file in test_file_names[:10]:
    test_img.append(cv2.imread(os.path.join(test_path, file)))
test_img = np.array(test_img)

# class types (I'm sure there's a better way of doing this)
galaxy_class_labels = np.array(["Class1.1", "Class1.2", "Class1.3", "Class2.1", "Class2.2", "Class3.1", "Class3.2", 
                "Class4.1", "Class4.2", "Class5.1", "Class5.2", "Class5.3", "Class5.4", "Class6.1",
                "Class6.2", "Class7.1", "Class7.2", "Class7.3", "Class8.1", "Class8.2", "Class8.3",
                "Class8.4", "Class8.5", "Class8.6", "Class8.7", "Class9.1", "Class9.2", "Class9.3",
                "Class10.1", "Class10.2", "Class10.3", "Class11.1", "Class11.2", "Class11.3", "Class11.4",
                "Class11.5", "Class11.6"])

In [7]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def reshape_arrays(data, labels):
    """reshape arrays for Keras"""
    data = data.reshape(-1, 424, 424, 3)
    labels = to_categorical(labels)
    return data, labels

# split the samples into training, validation and, test data sets:
train_frac = 0.8
val_frac = 0.2

# Note: we have to use train_test_split twice
data_train_val, data_test, class_train_val, class_test = train_test_split(training_img, training_img_sol, test_size=None)

data_train, data_val, class_train, class_val = train_test_split(
    training_img, training_img_sol, test_size=val_frac / (train_frac + val_frac)
)

class_train = np.asarray(class_train, dtype=np.float64)

'''The following commented-out code throws an error'''
#data_train, class_train = reshape_arrays(data_train, class_train)
#data_val, class_val = reshape_arrays(data_val, class_val)
#data_test, class_test = reshape_arrays(data_test, class_test)

print(
    f"Number of samples in the training ({data_train.shape[0]}); test ({data_train.shape[0]}); and validation ({data_val.shape[0]}) data sets"
)

2023-03-14 21:26:35.835711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 21:26:37.009325: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.9/site-packages/cv2/../../lib64:/opt/conda/pkgs/cudnn-8.1.0.77-h90431f1_0/lib/:/opt/conda/pkgs/cudatoolkit-11.2.2-he111cf0_8/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-14 21:26:37.009467: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot o

Number of samples in the training (1200); test (1200); and validation (300) data sets


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.utils import to_categorical

'''VGG6'''
def vgg6(input_shape=(424, 424, 3), n_classes: int = 37):
    """
        VGG6
    :param input_shape:
    :param n_classes:
    :return:
    """

    model = Sequential(name="VGG6")
    # input: 21x21 images with 1 channel -> (21, 21, 1) tensors.
    # this applies 16 convolution filters of size 3x3 each.
    model.add(Conv2D(16, (3, 3), activation="relu", input_shape=input_shape, name="conv1"))
    model.add(Conv2D(16, (3, 3), activation="relu", name="conv2"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), activation="relu", name="conv3"))
    model.add(Conv2D(32, (3, 3), activation="relu", name="conv4"))
    model.add(BatchNormalization(axis=3, name="bn_2"))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(256, activation="relu", name="fc_1"))
    model.add(Dropout(0.5))
    # output layer
    activation = "sigmoid" if n_classes == 1 else "softmax"
    model.add(Dense(n_classes, activation=activation, name="fc_out"))

    return model

In [9]:
'''RESNET50'''
## Create Neural Network w/ Keras (ResNet50)
# Based on https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/Residual_Networks_yourself.ipynb
from tensorflow.keras.layers import (
    Input,
    Add,
    Dense,
    Activation,
    ZeroPadding2D,
    BatchNormalization,
    Flatten,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
    GlobalMaxPooling2D,
)
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")


def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(
        filters=F1,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="valid",
        name=conv_name_base + "2a",
        kernel_initializer=glorot_uniform(seed=0),
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(
        filters=F2,
        kernel_size=(f, f),
        strides=(1, 1),
        padding="same",
        name=conv_name_base + "2b",
        kernel_initializer=glorot_uniform(seed=0),
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(
        filters=F3,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="valid",
        name=conv_name_base + "2c",
        kernel_initializer=glorot_uniform(seed=0),
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2c")(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X


def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_name_base + "2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(
        filters=F2,
        kernel_size=(f, f),
        strides=(1, 1),
        padding="same",
        name=conv_name_base + "2b",
        kernel_initializer=glorot_uniform(seed=0),
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(
        filters=F3,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="valid",
        name=conv_name_base + "2c",
        kernel_initializer=glorot_uniform(seed=0),
    )(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2c")(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(
        filters=F3,
        kernel_size=(1, 1),
        strides=(s, s),
        padding="valid",
        name=conv_name_base + "1",
        kernel_initializer=glorot_uniform(seed=0),
    )(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + "1")(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X


def ResNet50(input_shape=(424, 424, 3), classes=37):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name="conv1", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block="a", s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block="b")
    X = identity_block(X, 3, [64, 64, 256], stage=2, block="c")

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block="a", s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block="b")
    X = identity_block(X, 3, [128, 128, 512], stage=3, block="c")
    X = identity_block(X, 3, [128, 128, 512], stage=3, block="d")

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block="a", s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block="b")
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block="c")
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block="d")
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block="e")
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block="f")

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block="a", s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block="b")
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block="c")

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc" + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name="ResNet50")

    return model

In [10]:
final_model = vgg6() # for now choose between vgg6() and ResNet50()
final_model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
final_model.summary()

Model: "VGG6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 422, 422, 16)      448       
                                                                 
 conv2 (Conv2D)              (None, 420, 420, 16)      2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 210, 210, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 210, 210, 16)      0         
                                                                 
 conv3 (Conv2D)              (None, 208, 208, 32)      4640      
                                                                 
 conv4 (Conv2D)              (None, 206, 206, 32)      9248      
                                                              

2023-03-14 21:26:39.479803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 21:26:40.327063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10405 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [11]:
# Train model for 20 epochs
n_epochs = 20
batch_size = 128

final_model_history = final_model.fit(
    data_train,
    class_train,
    epochs=n_epochs,
    batch_size=batch_size,
    verbose=1,
    validation_data=None,
    shuffle=True,
)
classes = final_model.predict(data_test)

Epoch 1/20


2023-03-14 21:26:42.460500: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inVGG6/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-03-14 21:26:43.989589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-14 21:26:44.539448: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-14 21:26:46.781888: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be

 1/10 [==>...........................] - ETA: 1:16 - loss: 2.1962 - accuracy: 0.0234

2023-03-14 21:26:49.797775: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


10/10 [==============================] - 15s 750ms/step - loss: 1.1308 - accuracy: 0.1558
Epoch 2/20
10/10 [==============================] - 6s 622ms/step - loss: 0.6546 - accuracy: 0.1525
Epoch 3/20
10/10 [==============================] - 6s 611ms/step - loss: 0.6329 - accuracy: 0.1383
Epoch 4/20
10/10 [==============================] - 6s 599ms/step - loss: 0.5652 - accuracy: 0.2333
Epoch 5/20
10/10 [==============================] - 6s 579ms/step - loss: 0.5375 - accuracy: 0.2683
Epoch 6/20
10/10 [==============================] - 6s 590ms/step - loss: 0.5059 - accuracy: 0.2908
Epoch 7/20
10/10 [==============================] - 6s 617ms/step - loss: 0.4800 - accuracy: 0.3133
Epoch 8/20
10/10 [==============================] - 6s 592ms/step - loss: 0.4698 - accuracy: 0.3300
Epoch 9/20
10/10 [==============================] - 6s 595ms/step - loss: 0.4508 - accuracy: 0.3217
Epoch 10/20
10/10 [==============================] - 6s 596ms/step - loss: 0.4364 - accuracy: 0.3350
Epoch 11/

In [20]:
class_train[0,:], class_train[26,:], class_train[135,:], class_train[6,:], class_train.shape

(array([0.531033  , 0.389839  , 0.079128  , 0.        , 0.389839  ,
        0.        , 0.389839  , 0.20845822, 0.18138078, 0.        ,
        0.19650264, 0.12891274, 0.06442323, 0.039942  , 0.960058  ,
        0.34028754, 0.19074546, 0.        , 0.        , 0.        ,
        0.        , 0.039942  , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.20845822, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20845822]),
 array([0.713851  , 0.268749  , 0.0174    , 0.        , 0.268749  ,
        0.12199377, 0.14675523, 0.13818455, 0.13056445, 0.        ,
        0.08537618, 0.18337282, 0.        , 0.322795  , 0.677205  ,
        0.        , 0.66809529, 0.04575571, 0.18431595, 0.        ,
        0.04615968, 0.        , 0.04615968, 0.04615968, 0.        ,
        0.        , 0.        , 0.        , 0.13818455, 0.        ,
        0.        , 0.        , 0.13818455, 0.        , 0.        ,
        0.    